In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

In [12]:
transform = transforms.Compose([ transforms.ToTensor(),])
train_dataset = datasets.MNIST(root="./data", train=True, download=False,transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=False,transform=transform)
train_subset = Subset(train_dataset, range(200))
test_subset = Subset(test_dataset, range(50))
train_loader = DataLoader(train_subset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=10, shuffle=False)

In [16]:
class SimpleANN(nn.Module):
    
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear( 128,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32, 10)
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = SimpleANN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [18]:
def train_model(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct_train = 0
        total_train = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        predicted = torch.argmax(output.data, dim=1)
        total_train += target.size(0)
        correct_train += (predicted==target).sum().item()
    avg_train_loss = train_loss/len(train_loader)
    train_acc = 100 * correct_train/total_train
    model.eval()
    test_loss = 0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)

    loss = criterion(output, target)
    test_loss += loss.item()
    predicted = torch.argmax(output.data, dim=1)
    total_test += target.size(0)
    correct_test += (predicted==target).sum().item()
    
    avg_test_loss = test_loss/len(test_loader)
    test_acc = 100 * correct_test/total_test
    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_acc:.8f}%, ' f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.8f}%')
    
train_model(10)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x784 and 128x64)